<a href="https://colab.research.google.com/github/agrawalharish1997/sentiment-analysis/blob/master/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
import os
import re
import string
from sklearn.pipeline import Pipeline

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from sklearn.metrics import accuracy_score
from nltk.tokenize import sent_tokenize, word_tokenize,TweetTokenizer
from nltk import pos_tag 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,LabelBinarizer

In [5]:
data=pd.read_csv('train.csv')
df=pd.DataFrame(data,columns = ['id', 'label','tweet'])
Y=df.label
X=df.drop(['label','id'],axis=1)
#X["tweet"] = X["tweet"].str.lower()
#special_char_list = [':','+','#','$','@','!','&','%',';','/',',', '?', '}', ')', '{', '(','>','<']
#for special_char in special_char_list:
#    X["tweet"]=X["tweet"].replace(special_char,'')
#X["tweet"].head()
def word_count(sentence):
    return len(sentence.split())    
X['word count'] = X['tweet'].apply(word_count)
X.head(3)
def processTweet(tweet):
    tweet = re.sub(r'\&\w*;', '', tweet)
    tweet = re.sub('@[^\s]+','',tweet)
    tweet = re.sub(r'\$\w*', '', tweet)
    tweet = tweet.lower()
    tweet = re.sub(r'https?:\/\/.*\/\w*', '', tweet)
    tweet = re.sub(r'#\w!*', '', tweet)
    tweet = re.sub(r'[' + string.punctuation.replace('@', '') + ']+', ' ', tweet)
    tweet = re.sub(r'\b\w{1,2}\b', '', tweet)
    tweet = re.sub(r'\s\s+', ' ', tweet)
    tweet = tweet.lstrip(' ') 
    tweet = ''.join(c for c in tweet if c <= '\uFFFF') 
    return tweet
X['tweet'] = X['tweet'].apply(processTweet)
X['tweet'].head()


0    when father dysfunctional and selfish drags hi...
1    thanks for yft credit can use cause they don o...
2                                  bihday your majesty
3    odel love take with all the time urð± ...
4                     factsguide society now otivation
Name: tweet, dtype: object

In [0]:
all_words = []
for line in list(X['tweet']):
    words = line.split()
    for word in words:
        all_words.append(word.lower())

In [10]:
nltk.download('stopwords')
def text_process(raw_text):
    nopunc = [char for char in list(raw_text) if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    return [word for word in nopunc.lower().split() if word.lower() not in stopwords.words('english') and len(word)>3]
def remove_words(word_list):
    remove = ['paul','ryan','...','“','”','’','…','ryan’']
    return [w for w in word_list if w not in remove]
X = X.copy()
X['token'] = X['tweet'].apply(text_process) 
X['no_pauls'] = X['token'].apply(remove_words) 
X.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,tweet,word count,token,no_pauls
0,when father dysfunctional and selfish drags hi...,18,"[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunctional, selfish, drags, kids, ..."
1,thanks for yft credit can use cause they don o...,19,"[thanks, credit, cause, offer, wheelchair, van...","[thanks, credit, cause, offer, wheelchair, van..."
2,bihday your majesty,3,"[bihday, majesty]","[bihday, majesty]"
3,odel love take with all the time urð± ...,14,"[odel, love, take, time, urð±, , ...","[odel, love, take, time, urð±, , ..."
4,factsguide society now otivation,4,"[factsguide, society, otivation]","[factsguide, society, otivation]"


In [11]:
stem=PorterStemmer()
def stem_sentences(sentence):
    tokens = sentence
    stemmed_tokens = [stem.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

X['stemword'] =X["token"].apply(stem_sentences)
X['stemword'].head()  
X.head()

,tweet,word count,token,no_pauls,stemword
0,when father dysfunctional and selfish drags hi...,18,"[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunctional, selfish, drags, kids, ...",father dysfunct selfish drag kid dysfunct
1,thanks for yft credit can use cause they don o...,19,"[thanks, credit, cause, offer, wheelchair, van...","[thanks, credit, cause, offer, wheelchair, van...",thank credit caus offer wheelchair van isapoin...
2,bihday your majesty,3,"[bihday, majesty]","[bihday, majesty]",bihday majesti
3,odel love take with all the time urð± ...,14,"[odel, love, take, time, urð±, , ...","[odel, love, take, time, urð±, , ...",odel love take time urð±  ¦¦¦
4,factsguide society now otivation,4,"[factsguide, society, otivation]","[factsguide, society, otivation]",factsguid societi otiv


In [13]:
word=pd.get_dummies(X['stemword']).values
added_dummy_cols = [item for item in list(X['stemword'].values) ]
print(word)
print(word.shape)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(31962, 28033)


In [0]:
X_train, X_test, y_train, y_test =train_test_split(word,Y, test_size=0.3, random_state=42)
clf = LogisticRegression(random_state=0).fit(X_train,y_train)
y_pred=clf.predict(X_test)

In [15]:
accuracy_score(y_test,y_pred)
conf_Matrix = confusion_matrix(y_test,y_pred)
print(conf_Matrix)
print(accuracy_score(y_test,y_pred))

[[8905    0]
 [ 624   60]]
0.9349254353947232
